# Predictive Modeling
---

## Libraries

In [3]:
import sklearn
import pandas as pd

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

from sklearn import tree

from sklearn.ensemble import RandomForestClassifier

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import KFold

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn import cross_validation 
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict

## Import Data and Repeat Data Cleanup

In [5]:
bank_marketing = pd.read_csv("bank_marketing.csv")
processed_bank_marketing = bank_marketing.drop(columns=['default', 'poutcome', 'contact', 'month'])
processed_bank_marketing['total_contact'] = processed_bank_marketing['previous'] + processed_bank_marketing['campaign']
processed_bank_marketing = processed_bank_marketing.drop(columns=['previous', 'campaign', 'pdays'])
binary_response_map = {'yes':1, 'no':0}
processed_bank_marketing['success'] = processed_bank_marketing['y'].map(binary_response_map)
processed_bank_marketing['housing'] = processed_bank_marketing['housing'].map(binary_response_map)
processed_bank_marketing['loan'] = processed_bank_marketing['loan'].map(binary_response_map)
processed_bank_marketing = processed_bank_marketing.drop(columns=['y'])
processed_bank_marketing.head(5)

,age,job,marital,education,balance,housing,loan,day,duration,total_contact,success
0,30,unemployed,married,primary,1787,0,0,19,79,1,0
1,33,services,married,secondary,4789,1,1,11,220,5,0
2,35,management,single,tertiary,1350,1,0,16,185,2,0
3,30,management,married,tertiary,1476,1,1,3,199,4,0
4,59,blue-collar,married,secondary,0,1,0,5,226,1,0


## Train Model

In [20]:
test_size = 0.2
seed = 5
data = processed_bank_marketing.values
X = data[:,-4:-2] # Features
X
Y = data[:, -1] # Dependent Variable
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)
dt2 = tree.DecisionTreeClassifier(random_state=1, max_depth=2)
dt2.fit(X_train, y_train)
dt2_score_train = dt2.score(X_train, y_train)
print("Training score: ",dt2_score_train)
dt2_score_test = dt2.score(X_test, y_test)
print("Testing score: ",dt2_score_test)

ValueError: Unknown label type: 'unknown'